# Обучение трансформера, который будет генерировать шутки

Датасет взят с https://www.kaggle.com/datasets/konstantinalbul/russian-jokes

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM

import re
import os
import random

c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загружаем CSV-файл
df = pd.read_csv("jokes.csv", header=None)

jokes = df[1].dropna().tolist()  # Очистка от пустых строк
print(f"Всего шуток: {len(jokes)}")

Всего шуток: 130205


In [31]:
df

,0,1,2
0,theme,text,rating
1,pro-sudey,На суде в Стамбуле обвиняемый сказал:\r\n- На...,5
2,pro-sudey,"- Вы продолжаете утверждать, что обвиняемый н...",4
3,pro-sudey,"На суде.\r\n- Итак, когда дело дошло до столкн...",0
4,pro-sudey,Старую леди сбил автомобиль. На суде ее спраши...,4
...,...,...,...
130200,raznie,"Збежал медведь из зоопарка, ну передали по рад...",0
130201,raznie,"Разговаривают два грузина, Гоги и Авас:\r\nГ: ...",0
130202,raznie,В каждом из нас спит гений и с каждым днем вс...,0
130203,raznie,Очередь... Последней в ней стоит бабка - гор...,0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130205 entries, 0 to 130204
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       130205 non-null  object
 1   1       130205 non-null  object
 2   2       130205 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


In [3]:
from sklearn.model_selection import train_test_split
def preprocess_text(text):
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    # Удаляем спецсимволы, кроме разрешённых
    text = re.sub(r'[^а-яё\s.,!?]', '', text)
    return text

# Применяем предобработку ко всем шуткам
cleaned_jokes = [preprocess_text(joke) for joke in jokes]

train_texts, val_texts = train_test_split(cleaned_jokes, test_size=0.1, random_state=42)

In [5]:
# Создаем токенизатор
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3medium_based_on_gpt2")

# Расширяем словарь токенизатора, добавляя уникальные слова из наших шуток
special_tokens = {
    "bos_token": "<|startoftext|>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>"
}
tokenizer.add_special_tokens(special_tokens)

# Токенизируем все шутки
tokenized_train = tokenizer(train_texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
tokenized_val = tokenizer(val_texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

In [ ]:
class JokeDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone() 
        return item

# Создаем экземпляр датасета
train_dataset = JokeDataset(tokenized_train)
val_dataset = JokeDataset(tokenized_val)

In [7]:
# Создаем загрузчик данных
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [8]:
def evaluate(model, val_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    print(f"Валидационная потеря: {avg_loss:.4f}")
    return avg_loss

In [ ]:
# Загружаем предобученную модель и адаптируем её под наш токенизатор и переводим модель на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model.to(device)
model.resize_token_embeddings(len(tokenizer))  # Обновляем размер эмбеддингов под наш словарь

Embedding(50258, 768)

In [10]:
# Оптимизатор
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Функция потерь — перекрестная энтропия
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

In [ ]:
def generate_joke(prompt=""):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=128,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated.strip()

In [12]:
import time  # Для измерения времени
# Число эпох
epochs = 15

for epoch in range(epochs):
    
    # Режим обучения
    model.train()
    
    print(f"\nЭпоха {epoch + 1}/{epochs}")
    total_loss = 0
    
    start_time_epoch = time.time()  # Время начала эпохи

    for batch_idx, batch in enumerate(train_dataloader):
        start_time_batch = time.time()  # Время начала батча
        # Переносим данные на устройство
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        # Оценка времени на один батч
        batch_duration = time.time() - start_time_batch
        
        # Примерное оставшееся время для этой эпохи (в секундах)
        remaining_batches = len(train_dataloader) - batch_idx - 1
        estimated_remaining_time = batch_duration * remaining_batches

        # Форматируем вывод времени
        minutes = int(estimated_remaining_time // 60)
        seconds = int(estimated_remaining_time % 60)

        # Показываем статус каждые 50 батчей
        if (batch_idx + 1) % 50 == 0 or (batch_idx + 1) == len(train_dataloader):
            print(f"Батч {batch_idx + 1}/{len(train_dataloader)} | "
                  f"Потеря: {loss.item():.4f} | "
                  f"Осталось до конца эпохи: {minutes} мин {seconds} сек")

    avg_loss = total_loss / len(train_dataloader)
    epoch_duration = time.time() - start_time_epoch
    print(f"Средняя потеря за эпоху: {avg_loss:.4f}| " f"Время эпохи: {int(epoch_duration // 60)} мин {int(epoch_duration % 60)} сек")
    
    # Проверяем на валидации
    model.eval()
    val_loss = evaluate(model, val_dataloader, device)
    print(f"Пример шутки:")
    print(generate_joke("Почему"))


Эпоха 1/15


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Батч 50/7324 | Потеря: 1.7288 | Осталось до конца эпохи: 39 мин 22 сек
Батч 100/7324 | Потеря: 1.5212 | Осталось до конца эпохи: 39 мин 55 сек
Батч 150/7324 | Потеря: 1.5005 | Осталось до конца эпохи: 39 мин 31 сек
Батч 200/7324 | Потеря: 1.3462 | Осталось до конца эпохи: 40 мин 2 сек
Батч 250/7324 | Потеря: 1.6165 | Осталось до конца эпохи: 39 мин 36 сек
Батч 300/7324 | Потеря: 1.0691 | Осталось до конца эпохи: 40 мин 35 сек
Батч 350/7324 | Потеря: 1.3867 | Осталось до конца эпохи: 41 мин 16 сек
Батч 400/7324 | Потеря: 1.7230 | Осталось до конца эпохи: 39 мин 36 сек
Батч 450/7324 | Потеря: 1.5387 | Осталось до конца эпохи: 39 мин 38 сек
Батч 500/7324 | Потеря: 1.8679 | Осталось до конца эпохи: 39 мин 15 сек
Батч 550/7324 | Потеря: 1.3493 | Осталось до конца эпохи: 40 мин 57 сек
Батч 600/7324 | Потеря: 1.5011 | Осталось до конца эпохи: 40 мин 27 сек
Батч 650/7324 | Потеря: 1.2159 | Осталось до конца эпохи: 40 мин 23 сек
Батч 700/7324 | Потеря: 1.0736 | Осталось до конца эпохи: 40 мин 6

In [14]:
# Сохранение модели
model.save_pretrained("my_model")

# Сохранение токенизатора
tokenizer.save_pretrained("my_model")

('my_rugpt3_model\\tokenizer_config.json',
 'my_rugpt3_model\\special_tokens_map.json',
 'my_rugpt3_model\\vocab.json',
 'my_rugpt3_model\\merges.txt',
 'my_rugpt3_model\\added_tokens.json',
 'my_rugpt3_model\\tokenizer.json')

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("my_model_rl").to(device)
tokenizer = AutoTokenizer.from_pretrained("my_model_rl")

In [15]:
from transformers import StoppingCriteria, StoppingCriteriaList

class EndOfJoke(StoppingCriteria):
    def __init__(self, stop_token="!"):
        self.stop_token = stop_token
    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_token in tokenizer.decode(input_ids[0])

stopping_criteria = StoppingCriteriaList([EndOfJoke()])

In [17]:
# Переключаем модель в режим оценки
model.eval()

# Функция генерации шутки
def generate_joke(prompt="Чем больше детей в подвалах корейских учителей"):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length= 128,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        early_stopping=True,
        stopping_criteria=stopping_criteria,
        num_beams=5,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated_joke = tokenizer.decode(output[0], skip_special_tokens=False)
    return generated_joke

# Пример генерации
print("\nПример сгенерированной шутки:")
print(generate_joke())


Пример сгенерированной шутки:
Чем больше детей в подвалах корейских учителей, тем лучше видны следы борьбы между школами в югославии и китаицами. особенно сильное влияние на эти сцены дает прочтение свитка справа налево сразу за решеткои слева направо, где по обе стороны полярнои звезды на краю кровавои линии между двумя львами, растущими из кровавым пятном на центре пятна на листке справа. этим магическим узором магического вихря убившеи звезде сверху падает на землю. мертвые львы склонились над мертвои косо


# Дообучение с Reinforcement Learning для улучшения качества генерации

In [ ]:
# Импорт необходимных библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import random
import re
import time
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка токенизатора и модели из папки my_model
tokenizer = AutoTokenizer.from_pretrained("my_model")
model = AutoModelForCausalLM.from_pretrained("my_model").to(device)

# Фиксируем веса оригинальной модели как "старую политику"
old_model = AutoModelForCausalLM.from_pretrained("my_model").to(device)
old_model.eval()  # Переводим в режим оценки, чтобы не обновлялись веса

# Функция предобработки текста: нормализация, удаление лишних символов
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^а-яё\s.,!?]', '', text)  # Оставляем только русские буквы, знаки препинания и пробелы
    return text

# Функция генерации шутки с заданным промптом
from transformers import StoppingCriteria, StoppingCriteriaList

class EndOfJoke(StoppingCriteria):
    def __init__(self, stop_token="!"):
        self.stop_token = stop_token
    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_token in tokenizer.decode(input_ids[0])

stopping_criteria = StoppingCriteriaList([EndOfJoke()])

def generate_joke(prompt="Почему"):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length= 64,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        early_stopping=True,
        stopping_criteria=stopping_criteria,
        num_beams=5,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated_joke = tokenizer.decode(output[0], skip_special_tokens=False)
    return generated_joke

# Пример использования генерации до RL
print("Пример шутки до RL:")
print(generate_joke())



# Реализация метрики BLEU
def sentence_bleu(reference, hypothesis, n=2):
    bleu_score = 0.0
    weights = [1 / n] * n

    count_matches = 0
    count_total = 0
    for ngram in range(1, n + 1):
        ref_ngrams = set()
        hyp_ngrams = set()

        for ref in reference:
            for i in range(len(ref) - ngram + 1):
                ref_ngrams.add(tuple(ref[i:i + ngram]))

        for i in range(len(hypothesis) - ngram + 1):
            hyp_ngrams.add(tuple(hypothesis[i:i + ngram]))

        matches = ref_ngrams.intersection(hyp_ngrams)
        count_matches += len(matches)
        count_total += len(hyp_ngrams)

    precision = count_matches / count_total if count_total != 0 else 0.0

    brevity_penalty = 1.0
    ref_lengths = [len(ref) for ref in reference]
    closest_ref_len = min(ref_lengths, key=lambda x: abs(x - len(hypothesis)))
    if len(hypothesis) < closest_ref_len:
        brevity_penalty = np.exp(1 - closest_ref_len / len(hypothesis))

    bleu_score = brevity_penalty * precision
    return bleu_score

# Reward-функция на основе BLEU
def calculate_reward(generated_text, reference_texts):
    tokenized_generated = generated_text.split()
    tokenized_references = [text.split() for text in reference_texts]
    reward = sentence_bleu(tokenized_references, tokenized_generated)
    return reward

# PPO Loss Function
def ppo_loss(logits, old_logits, actions, advantages, epsilon=0.2):
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    old_log_probs = torch.nn.functional.log_softmax(old_logits, dim=-1)

    selected_log_probs = log_probs.gather(-1, actions.unsqueeze(-1)).squeeze(-1)
    selected_old_log_probs = old_log_probs.gather(-1, actions.unsqueeze(-1)).squeeze(-1)

    ratio = torch.exp(selected_log_probs - selected_old_log_probs)
    surr1 = ratio * advantages
    surr2 = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * advantages
    loss = -torch.min(surr1, surr2).mean()

    return loss

# Датасет для RL обучения
class JokesRLDataset(Dataset):
    def __init__(self, jokes, tokenizer, max_length=64):
        self.jokes = jokes
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.jokes)

    def __getitem__(self, idx):
        joke = self.jokes[idx]
        encoding = self.tokenizer(
            joke,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Загрузка датасета
df = pd.read_csv("jokes.csv", header=None)
jokes = df[1].dropna().apply(preprocess_text).tolist()
jokes = random.sample(jokes, 25000)

# Разделение на тренировочные и валидационные данные
train_jokes, val_jokes = train_test_split(jokes, test_size=0.1, random_state=42)

# Создание датасетов и загрузчиков
train_dataset = JokesRLDataset(train_jokes, tokenizer)
val_dataset = JokesRLDataset(val_jokes, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

# Оптимизатор и планировщик
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 5)

# PPO Training Loop
epochs = 5
ppo_epochs = 2
epsilon = 0.2

for epoch in range(epochs):
    model.train()
    total_loss = 0
    start_time_epoch = time.time()

    print(f"\nЭпоха {epoch+1}/{epochs}")

    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        _, actions = torch.topk(logits, k=1, dim=-1)
        actions = actions.squeeze(-1)

        # Генерируем текст и вычисляем reward
        generated_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in actions]
        rewards = []
        for gen_text in generated_texts:
            sample_references = np.random.choice(train_jokes, size=3).tolist()
            reward = calculate_reward(gen_text, sample_references)
            rewards.append(reward)

        # Преобразуем в тензор и нормализуем
        advantages = torch.tensor(rewards, device=device)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        advantages = advantages.unsqueeze(-1).expand_as(actions)

        # Обновление модели через PPO
        for _ in range(ppo_epochs):
            optimizer.zero_grad()

            new_logits = model(input_ids=input_ids, attention_mask=attention_mask).logits

            with torch.no_grad():
                old_logits = old_model(input_ids=input_ids, attention_mask=attention_mask).logits

            loss = ppo_loss(new_logits, old_logits, actions, advantages, epsilon)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        # Обновление старой модели
        if batch_idx % 10 == 0:
            old_model.load_state_dict(model.state_dict())

        # Счётчик времени до конца эпохи
        current_time = time.time()
        elapsed = current_time - start_time_epoch
        avg_time_per_batch = elapsed / (batch_idx + 1)
        remaining_batches = len(train_loader) - batch_idx - 1
        remaining_time = avg_time_per_batch * remaining_batches

        mins = int(remaining_time // 60)
        secs = int(remaining_time % 60)

        print(f"Батч {batch_idx}/{len(train_loader)} | Потеря: {loss.item():.4f} | Осталось до конца эпохи: {mins} мин {secs} сек")

    avg_loss = total_loss / len(train_loader)
    duration = time.time() - start_time_epoch
    print(f"Эпоха завершена | Средняя потеря: {avg_loss:.4f} | Время: {int(duration // 60)} мин {int(duration % 60)} сек")

# Сохранение финальной модели
model.save_pretrained("my_model_rl")
tokenizer.save_pretrained("my_model_rl")

# Проверка качества после RL
print("\nПример шутки после RL:")
print(generate_joke())

c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Пример шутки до RL:
Почему в россииском шоубизнесе так много ремеиков и режиссеров?  потому что без труда не рождаются хорошие певцы и не становятся плохими танцорами

Эпоха 1/5
Батч 0/5625 | Потеря: 0.0387 | Осталось до конца эпохи: 73 мин 51 сек
Батч 1/5625 | Потеря: 0.1339 | Осталось до конца эпохи: 64 мин 0 сек
Батч 2/5625 | Потеря: 0.0662 | Осталось до конца эпохи: 61 мин 38 сек
Батч 3/5625 | Потеря: 0.0235 | Осталось до конца эпохи: 59 мин 38 сек
Батч 4/5625 | Потеря: 0.0489 | Осталось до конца эпохи: 58 мин 41 сек
Батч 5/5625 | Потеря: 0.0686 | Осталось до конца эпохи: 58 мин 2 сек
Батч 6/5625 | Потеря: 0.0386 | Осталось до конца эпохи: 58 мин 5 сек
Батч 7/5625 | Потеря: 0.0723 | Осталось до конца эпохи: 57 мин 41 сек
Батч 8/5625 | Потеря: 0.0924 | Осталось до конца эпохи: 57 мин 44 сек
Батч 9/5625 | Потеря: 0.0747 | Осталось до конца эпохи: 57 мин 32 сек
Батч 10/5625 | Потеря: 0.2062 | Осталось до конца эпохи: 57 мин 23 сек
Батч 11/5625 | Потеря: 0.0787 | Осталось до конца эпох

# Вывод

## Общая информация

Была реализована модель на основе трансформера, обученная генерировать текстовые шутки и улучшенная с помощью метода **Reinforcement Learning (PPO)**.

Использовался датасет из ~130 тысяч русскоязычных шуток, взятых с Kaggle. Для обучения применялась предобученная языковая модель `ai-forever/rugpt3small_based_on_gpt2`. Также были выполнены этапы предобработки данных, создания кастомного токенизатора, обучения модели и последующего дообучения с использованием RL.

---

## Результаты обучения без RL

Модель показала неплохие начальные результаты в генерации текста:

- **Уменьшение потерь** за 15 эпох: с 1.25 до 0.54
- **Примеры генерации**:
  - *"Почему евреи не пьют водку? Потому что они считают, что это понедельник."*
  - *"Почему блондинкам нельзя стричь волосы? Они начинают задыхаться."*

Текст был структурирован, логичен и соответствовал типичной структуре анекдотов, однако часто генерировался до максимально возможной длины и в ходе генерации терял смысл. В последствии генерация текста была улучшена с помощью StoppingCriteria, StoppingCriteriaList из библиотеки transformers

---

## Дообучение с использованием Reinforcement Learning (PPO)

### Подход:
- Реализован алгоритм **Proximal Policy Optimization (PPO)**
- Использована метрика **BLEU** как reward-функция
- Обновление весов модели происходило с учетом отношения вероятностей между новой и старой моделью
- Цель: улучшить качество генерации текста, сделать его более осмысленным и близким к эталонным примерам

### Результаты:
- **Средняя потеря** постепенно снижалась:
  - Эпоха 1: 478,884
  - Эпоха 5: 2.2166
- **Пример генерации после RL**:
  - *"Почему боярской до тех пор, пока не нашли своего павлика, который улетел за чернильником дровосека господня покойного князя Игоря?"*

Хотя полный смысл шутки, как шутки, не сохранялся, текст стал более связным и близким к литературной форме, чем до RL.

---

## Сравнение качества генерации

| Критерий                  | До RL                              | После RL                             |
|---------------------------|-------------------------------------|--------------------------------------|
| Логичность                | Хорошая                             | Улучшилась                          |
| Связность                 | Высокая                             | Стала стабильнее                    |
| Оригинальность            | Средняя                             | Несколько повысилась                |
| Соответствие формату шутки| Отличное                            | Снизилось                           |

---

## Проблема с потерями во время RL-дообучения

На первой эпохе наблюдались **резкие скачки потери**, достигающие значений свыше **478,000**. Почитав источники я узнал, что это может быть связано с:

- **Нестабильностью PPO-алгоритма** при первых обновлениях
- **Не очень хорошей реализацией reward-функции (BLEU)** — метрика BLEU не всегда корректно отражает естественность текста
- **Высокой температурой генерации** — приводит к большому разбросу ответов
- **Малым размером batch'а** (всего 4 примера), что делает обучение менее стабильной

Модель можно было бы улучшить с помощью улучшения reward-функции, оптимизации архитектуры и гиперпараметров и других методов, но в силу нехватки времени было решено оставить уже всё, как есть.